In [1]:
import torch
import torch
import torch.nn as nn
from torch.nn import functional as F

c:\Users\Pc\miniconda3\envs\gait\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# with open("input.txt") as f:
with open("./cleaned_merged_fairy_tales_without_eos.txt") as f:
    text = f.read()



In [3]:
print("The length of total dataset: ",len(text))

The length of total dataset:  20455694


In [4]:
print("The first 100 characters of the dataset: \n",text[:100])

The first 100 characters of the dataset: 
 The Happy Prince.
HIGH above the city, on a tall column, stood the statue of the Happy Prince.  He w


In [5]:
char = sorted(set(text))

vocab_size = len(char)
print(char)
print(''.join(char))
print("The number of unique characters in the dataset: ",vocab_size)

['\n', ' ', '!', '"', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '\xa0', '£', '°', '½', 'Ä', 'Æ', 'É', 'à', 'â', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ñ', 'ô', 'ö', 'ü', 'ā', 'œ', '—', '‘', '’', '“', '”', '…']

 !"$&'()*+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]`abcdefghijklmnopqrstuvwxyz{|} £°½ÄÆÉàâäæçèéêëîïñôöüāœ—‘’“”…
The number of unique characters in the dataset:  117


In [6]:
stoi = {chi:i for i,chi in enumerate(char) }
itos = {i:chi for i,chi in enumerate(char) }

encoder = lambda x: [stoi[chari] for chari in x]
decoder = lambda x: ''.join([itos[chari] for chari in x])


print(encoder("hii there"))
print(decoder([46, 47, 47, 1, 58, 46, 43, 56, 43]))

print(''.join(['cd', 'e']))

[65, 66, 66, 1, 77, 65, 62, 75, 62]
RSS aRO]O
cde


In [7]:

data = torch.tensor(encoder(text))

n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8 # Context size
# decoder(train_data[:block_size+1].numpy())

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"The context is: {context} and the target is: {target}")

The context is: tensor([48]) and the target is: 65
The context is: tensor([48, 65]) and the target is: 62
The context is: tensor([48, 65, 62]) and the target is: 1
The context is: tensor([48, 65, 62,  1]) and the target is: 36
The context is: tensor([48, 65, 62,  1, 36]) and the target is: 58
The context is: tensor([48, 65, 62,  1, 36, 58]) and the target is: 73
The context is: tensor([48, 65, 62,  1, 36, 58, 73]) and the target is: 73
The context is: tensor([48, 65, 62,  1, 36, 58, 73, 73]) and the target is: 82


In [10]:
ix = torch.randint(8, (4,))
print(ix)

tensor([1, 5, 7, 2])


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split, device = 'cpu'):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 65,  62,  75,  76,   1,  76,  62,  77],
        [ 63,  66,  75,  62,  11,   1,  58,  71],
        [115,   0,  29,  77,   1,  77,  65,  62],
        [ 77,  65,   1,  58,  71,   1,  62,  81]])
targets:
torch.Size([4, 8])
tensor([[62, 75, 76,  1, 76, 62, 77,  1],
        [66, 75, 62, 11,  1, 58, 71, 61],
        [ 0, 29, 77,  1, 77, 65, 62,  1],
        [65,  1, 58, 71,  1, 62, 81, 73]])
----
when input is [65] the target: 62
when input is [65, 62] the target: 75
when input is [65, 62, 75] the target: 76
when input is [65, 62, 75, 76] the target: 1
when input is [65, 62, 75, 76, 1] the target: 76
when input is [65, 62, 75, 76, 1, 76] the target: 62
when input is [65, 62, 75, 76, 1, 76, 62] the target: 77
when input is [65, 62, 75, 76, 1, 76, 62, 77] the target: 1
when input is [63] the target: 66
when input is [63, 66] the target: 75
when input is [63, 66, 75] the target: 62
when input is [63, 66, 75, 62] the target: 11
when input is [63, 66, 75, 62, 1

In [12]:
import torch.nn as nn
position_embedding = nn.Embedding(6, 3) 
pos_embedd =   (torch.arange(6))
print(pos_embedd.shape)

torch.Size([6])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
eval_iters = 200
max_iters = 1000

n_embedding = 32





class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embedding) # n_embedding to make the model a little longer
        self.position_embedding = nn.Embedding(block_size, n_embedding) # n_embedding to make the model a little longer
        self.lm_head = nn.Linear(n_embedding, vocab_size)
    def forward(self, idx, targets=None):
        B,T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_embedd = self.token_embedding_table(idx) # (B,T,C)
        pos_embedd =  self.position_embedding(torch.arange(T))  # T,C
        x = pos_embedd + tok_embedd # B,T,C
        logits = self.lm_head(x) 
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution. only num_sample prediction from the logits 
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decoder(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 117])
tensor(5.0743, grad_fn=<NllLossBackward0>)


IndexError: index out of range in self

In [14]:
@torch.no_grad()
def estimateLoss(model, device = 'cpu'):
    out = {}
    # print(device)
    model = model.to(device)
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split, device)
            # print(x.device, y.device, model)
            x = x.to(device)
            y = y.to(device)
            model = model.to(device)
            logits, loss = model(x , y)
            losses[k] = loss.item()
        out[split] = losses.mean()
        model.train()
    return out

### Separate function for the loss, so to calculate the loss on the data

In [15]:
batch_size = 32
optimizer = torch.optim.Adam(m.parameters(), lr=0.001)
for  iter in range(max_iters):
    if iter % 100 == 0:
        losses = eval_iters()
        print(f"iter {iter}, train loss: {losses['train']}, val loss: {losses['val']}")
    
    xb, yb = get_batch('train')
    lotics, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        # print(decoder(m.generate(idx = xb[:1], max_new_tokens=100)[0].tolist()))


print(f"The loss is: {loss.item()}")


TypeError: 'int' object is not callable

In [16]:
print(decoder(m.generate(idx = xb[:4], max_new_tokens=100)[0].tolist()))

IndexError: index out of range in self

# To get some kind of context between the prediction, we will form transformers

## Query `What I am looking for`
## Key `What I contain`
## Value `What value I will coomunicate`

In [17]:
torch.manual_seed(1337)
B,T,C = 4,8,2

sample = torch.randn(   (B,T,C))
print(sample.shape)
print(sample)


torch.Size([4, 8, 2])
tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])


In [18]:
xbox = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xpre = sample[b,:t+1] # [t, C]
        xbox[b,t] = torch.mean(xpre, 0)

print(xbox.shape)


torch.Size([4, 8, 2])


# Math trick behind the Self-Attention

In [19]:
torch.manual_seed(42)

a = torch.tril(torch.ones(3,3))
print(a)
a = a/torch.sum(a, dim= 1)

b = torch.randint(0,10, (3, 2)).float()
c = a@b

print(a)
print(b)
print(c)


tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000],
        [1.0000, 0.5000, 0.0000],
        [1.0000, 0.5000, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[ 2.0000,  7.0000],
        [ 5.0000,  9.0000],
        [ 7.0000, 10.6667]])


In [20]:
print(xbox.shape)
x = torch.randn((B,T,C))

wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1, keepdim=True)
xbow2 = wei @ xbox # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbox,xbow2)


torch.Size([4, 8, 2])


False

In [21]:
tril = torch.tril(torch.ones(3,3))
print(tril)
wei = torch.ones(3,3)
print(wei)
# This means that the token from the future don't have any valu
wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei)

wei = F.softmax(wei, dim=1)
print(wei)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., -inf, -inf],
        [1., 1., -inf],
        [1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])


In [22]:
head_size = 16
x = torch.randn(B,T,C)
key = nn.Linear(C, head_size , bias = False)
query = nn.Linear(C, head_size , bias = False)
value = nn.Linear(C, head_size , bias = False)

k = key(x)
q = query(x)
wei = q@k.transpose(-2,-1) # (B,T,16) @ (B,16,T) ---- (B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0 , float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
print(v.shape)
out = wei @ v # (B,T,T) @ (B,T,16)


out.shape


torch.Size([4, 8, 16])


torch.Size([4, 8, 16])

## Bigger the value in the input of th esoftmax, more sharpen the value is

# Multihead `Communication and context find`
# Feedforward `Computation of the context`

In [23]:
temp = torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1) # gets too peaky, converges to one-hot
print(temp)
temp = torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot
print(temp)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


In [24]:
batch_size = 256 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embedding = 256
head_size = 4
n_layer = 4
dropout = 0.0


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embedding, head_size, bias=False)
        self.query = nn.Linear(n_embedding, head_size, bias=False)
        self.value = nn.Linear(n_embedding, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out
    

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.head = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embedding, n_embedding)
    def forward(self,x):
        # Concatenare over the channel dimension
        out = torch.cat([h(x) for h in self.head], dim=-1)
        out = self.proj(out)

        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    


class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel_simple(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embedding) # n_embedding to make the model a little longer
        self.position_embedding = nn.Embedding(block_size, n_embedding) # n_embedding to make the model a little longer
        # self.head = Head(n_embedding)
        self.m_head = MultiHeadAttention(4 , n_embedding//4)
        self.lm_head = nn.Linear(n_embedding, vocab_size)
        self.feed_forward = FeedFoward(n_embedding)
    def forward(self, idx, targets=None):
        B,T = idx.shape
        # print("Input: ", idx.shape)
        # idx and targets are both (B,T) tensor of integers
        tok_embedd = self.token_embedding_table(idx) # (B,T,C)
        # print("Token Embedding: ",tok_embedd.shape)
        pos_embedd =  self.position_embedding(torch.arange(T))  # T,C
        # print("Positional Embedding: ",pos_embedd.shape)
        x = pos_embedd + tok_embedd # B,T,C
        # print("Added Embedding: ",x.shape)
        x = self.m_head(x) # apply one head of the self-attention (B,T,C)
        x = self.feed_forward(x)
        # print("After single head: ",x.shape)
        logits = self.lm_head(x)  # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size
            # We cannot enter the embedding greater than batch_size
            # print("----------idx------------",idx.shape)
            idx_cond = idx[:, -block_size:]
            # print("----------idx_cond------------",idx_cond.shape)
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution. only num_sample prediction from the logits 
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            # print(idx.shape, idx_next.shape)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embedding) # n_embedding to make the model a little longer
        self.position_embedding = nn.Embedding(block_size, n_embedding) # n_embedding to make the model a little longer
        # self.head = Head(n_embedding)
        self.blocks = nn.Sequential(
            Block(n_embedding, n_head=4 ),
            Block(n_embedding, n_head=4 ),
            Block(n_embedding, n_head=4 ),
        )
        self.l_norm = nn.LayerNorm(n_embedding)
        self.lm_head = nn.Linear(n_embedding, vocab_size)
    def forward(self, idx, targets=None):
        B,T = idx.shape
        # print("Input: ", idx.shape)
        # idx and targets are both (B,T) tensor of integers
        tok_embedd = self.token_embedding_table(idx) # (B,T,C)
        # print("Token Embedding: ",tok_embedd.shape)
        pos_embedd =  self.position_embedding(torch.arange(T, device = device))  # T,C
        # print("Positional Embedding: ",pos_embedd.shape)
        x = pos_embedd + tok_embedd # B,T,C
        # print("Added Embedding: ",x.shape)
        x = self.blocks(x) # apply one head of the self-attention (B,T,C)
        # print("After single head: ",x.shape)
        x = self.l_norm(x)

        logits = self.lm_head(x)  # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size
            # We cannot enter the embedding greater than batch_size
            # print("----------idx------------",idx.shape)
            idx_cond = idx[:, -block_size:]
            # print("----------idx_cond------------",idx_cond.shape)
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution. only num_sample prediction from the logits 
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            # print(idx.shape, idx_next.shape)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
# logits, loss = model(xb, yb)
# print(logits.shape)
# print(loss)
model = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
# batch_size = 32
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for  iter in range(max_iters):
    if iter % 100 == 0:
        losses = estimateLoss(model, device = device)
        print(f"iter {iter}, train loss: {losses['train']}, val loss: {losses['val']}")
    
    xb, yb = get_batch('train', device)
    # xb = xb.to(device)
    # yb = yb.to(device)
    lotics, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        # print(decoder(m.generate(idx = xb[:1], max_new_tokens=100)[0].tolist()))


print(f"The loss is: {loss.item()}")






2.435701 M parameters
iter 0, train loss: 4.927412033081055, val loss: 4.926512718200684
iter 100, train loss: 2.151301622390747, val loss: 2.186861276626587
iter 200, train loss: 1.9328367710113525, val loss: 1.9917336702346802
iter 300, train loss: 1.8265316486358643, val loss: 1.8819069862365723
iter 400, train loss: 1.760615348815918, val loss: 1.8279606103897095
iter 500, train loss: 1.7130508422851562, val loss: 1.7766151428222656
iter 600, train loss: 1.6706608533859253, val loss: 1.7401058673858643
iter 700, train loss: 1.6392855644226074, val loss: 1.7151566743850708
iter 800, train loss: 1.6170307397842407, val loss: 1.6941725015640259
iter 900, train loss: 1.5942291021347046, val loss: 1.671135663986206
iter 1000, train loss: 1.5698051452636719, val loss: 1.653064250946045
iter 1100, train loss: 1.557676076889038, val loss: 1.642174482345581
iter 1200, train loss: 1.5463448762893677, val loss: 1.6270047426223755
iter 1300, train loss: 1.5334010124206543, val loss: 1.61668777

In [27]:
torch.save(model.state_dict(), "model.pt")

# #Later to restore:
# model.load_state_dict(torch.load(filepath))
# model.eval()

In [25]:
def get_answer():
    input_data = input("Please enter the prompt: ")
    encoded_data = encoder(input_data)
    print(encoded_data)
    encoded_data = torch.tensor(encoded_data)
    encoded_data = encoded_data[None, :]
    encoded_data = encoded_data.to(device)
    print(encoded_data.shape)
    out = model.generate(idx = torch.tensor(encoded_data[:]), max_new_tokens=100)[0].tolist()
    out = decoder(out)
    print(out)

get_answer()

[37, 1, 58, 70, 1, 64, 69, 58, 61, 1, 77, 65, 62, 75, 62, 1, 66, 76, 1, 72, 71, 62]
torch.Size([1, 22])


C:\Users\Pc\AppData\Local\Temp\ipykernel_1712\2116348823.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = model.generate(idx = torch.tensor(encoded_data[:]), max_new_tokens=100)[0].tolist()


I am glad there is one homelquareeal world in a read: He was sore got upon the looked against their and from the cortal yo


In [109]:
x = torch.randn(4,1,2)
print(x)
idx_cond = x[:, -32:]
print(idx_cond)


tensor([[[-0.0122,  0.9852]],

        [[-1.9529,  0.2810]],

        [[ 0.3200, -1.0520]],

        [[-0.6644,  0.0957]]])
tensor([[[-0.0122,  0.9852]],

        [[-1.9529,  0.2810]],

        [[ 0.3200, -1.0520]],

        [[-0.6644,  0.0957]]])


In [ ]:
eval_iters = 200
max_iters = 1000

n_embedding = 32


batch_size = 32
optimizer = torch.optim.Adam(m.parameters(), lr=0.001)
for  iter in range(max_iters):
    if iter % 100 == 0:
        losses = estimateLoss()
        print(f"iter {iter}, train loss: {losses['train']}, val loss: {losses['val']}")
    
    xb, yb = get_batch('train')
    lotics, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        # print(decoder(m.generate(idx = xb[:1], max_new_tokens=100)[0].tolist()))


print(f"The loss is: {loss.item()}")
